## Match up satellite and buoy data

In this exercise you will match satellite data and buoy data by extracting satellite data around a set of points defined by buoy locations.  

### The exercise demonstrates the following techniques:
- Loading satellite data in polar stereographic projection from ERDDAP
- Loading buoy data in geographical coordinate system (latitude, longitude) (csv format) from ERDDAP
- Match and extract satellite data along the buoy locatiions
- Plotting both buoy data values onto the map


### Data Used

__Ice Surface Temperature, NOAA-20 VIIRS, Near Real-Time, Polar Stereographic (North), Daily 4-day Composite__

https://polarwatch.noaa.gov/erddap/info/noaacwVIIRSn20icesrftempNP06Daily4Day/index.html


__International Arctic Buoy Programme (IABP) Buoy Data, Daily__

https://polarwatch.noaa.gov/erddap/info/iabpv2_buoys/index.html

International Arctic Buoy Programme meteorological and oceanographic buoys. Data are updated daily. The preview and download interfaces for this dataset are currently in beta.

In [4]:
import xarray as xr
import pandas as pd
import requests
import io



In [17]:
# load 

gridded_url = "https://polarwatch.noaa.gov/erddap/griddap"

satellite_id = "noaacwVIIRSn20icesrftempNP06Daily4Day"


sat_url = "/".join([gridded_url, satellite_id])


# # Select one day data
# ds_date = '2024-07-30'
# ds = ds.sel(time=ds_date)

# # Visualize the IMS data
# ds["IMS_Surface_Values"].plot()



In [18]:
ds_sat = xr.open_dataset(sat_url)
ds_sat

<xarray.Dataset> Size: 303GB
Dimensions:     (time: 1015, altitude: 1, rows: 8640, cols: 8640)
Coordinates:
  * time        (time) datetime64[ns] 8kB 2021-04-13 2021-04-14 ... 2024-08-31
  * altitude    (altitude) float64 8B 0.0
  * rows        (rows) float64 69kB 3.434e+06 3.433e+06 ... -3.434e+06
  * cols        (cols) float64 69kB -3.434e+06 -3.433e+06 ... 3.434e+06
Data variables:
    IceSrfTemp  (time, altitude, rows, cols) float32 303GB ...
Attributes: (12/36)
    cdm_data_type:                     Grid
    Conventions:                       CF-1.6, COARDS, ACDD-1.3
    creator_email:                     coastwatch.info@noaa.gov
    creator_name:                      NOAA/NESDIS/STAR/SOCD/CoastWatch
    creator_url:                       https://coastwatch.noaa.gov
    crs_wkt:                           PROJCS["WGS 84 / NSIDC Sea Ice Polar S...
    ...                                ...
    start_orbit_number:                35124
    summary:                           Enterprise Ice Concentration/ Ice Mask...
    testOutOfDate:                     now-2days
    time_coverage_end:                 2024-08-31T00:00:00Z
    time_coverage_start:               2021-04-13T00:00:00Z
    title:                             Ice Surface Temperature, NOAA-20 VIIRS...

### A note about tabledap

Most of our examples in this course use gridded datasets. The NDBC data for this tutorial is a tabular dataset, served via the tabledap part of ERDDAP. The API for tabledap is a little different than for gridded datasets. You can go to the following URL and play around with subsetting. Then push the "Just generate the URL" button, copy the link, put it in a browser. See if you get what you expected. https://coastwatch.pfeg.noaa.gov/erddap/tabledap/cwwcNDBCMet

A quick primer is below

The data request URL has three parts:

Base URL: https://url/erddap/tabledap/datasetID.fileType? 
e.g. https://coastwatch.pfeg.noaa.gov/erddap/tabledap/cwwcNDBCMet.csv?
A list of variables you want to download that are separated by commas
e.g. station,longitude,latitude,time,wtmp
A list of constraints, each starting with an ampersand (&).
The constraints use =, >, >=, <, and <= to subset the data
e.g. &station="46259", mean station # 46259
e.g. &time>=2017-01-01T&time<=2020-12-31', means time between and including Jan. 1, 2017 and Dec. 31, 2020.
The data request URL we will use for the NDBC data:
ndbc_url = 'https://coastwatch.pfeg.noaa.gov/erddap/tabledap/cwwcNDBCMet.csv?station,longitude,latitude,time,wtmp&station="46259"&time>=2017-01-01T&time<=2020-12-31

https://github.com/coastwatch-training/CoastWatch-Tutorials/blob/main/create-virtual-buoy-with-satellite-data/Python/create-virtual-buoy-with-satellite-data.ipynb

In [7]:
# Break the url into part and rejoin it so that it is easier to see.

# https://polarwatch.noaa.gov/erddap/tabledap/iabpv2_buoys.csv?time%2Clatitude%2Clongitude%2Csurface_temp%2Chas_surface_temp%2Cbuoy_id%2Cbuoy_type&time%3E=2024-08-29T00%3A00%3A00Z&time%3C=2024-09-05T14%3A50%3A29Z&has_surface_temp=%22yes%22


buoy_url = ''.join(['https://polarwatch.noaa.gov/erddap/tabledap/iabpv2_buoys.csv?',
                    'buoy_id,longitude,latitude,time,surface_temp',
                    '&has_surface_temp="yes"&time>=2023-09-04&time<=2023-09-05'
                    ])

req = requests.get(buoy_url).content
buoy_df = pd.read_csv(io.StringIO(req.decode('utf-8')), skiprows=[1], parse_dates=['time'])
buoy_df


,buoy_id,longitude,latitude,time,surface_temp
0,300234066034140,-25.75460,53.11580,2023-09-01 00:00:00+00:00,15.20
1,300234066034140,-25.75460,53.11580,2023-09-01 01:00:02+00:00,15.20
2,300234066034140,-25.75460,53.11580,2023-09-01 01:59:57+00:00,15.20
3,300234066034140,-25.71160,53.10440,2023-09-01 03:00:00+00:00,15.20
4,300234066034140,-25.71160,53.10440,2023-09-01 04:00:02+00:00,15.30
...,...,...,...,...,...
24609,300025060513230,114.28790,88.43560,2023-09-04 18:50:15+00:00,0.03
24610,300025060513230,114.28790,88.43560,2023-09-04 18:50:24+00:00,0.02
24611,300025060513230,114.28790,88.43560,2023-09-04 18:50:32+00:00,0.03
24612,300025060513230,114.28790,88.43560,2023-09-04 18:50:41+00:00,0.04
